<a href="https://colab.research.google.com/github/tokyo8182/LLM-RAG/blob/main/%5BMemoona%5D_Llamaparse_%2B_groq_pipeline_for_metadata_for_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Read in all files i.e. data ingestion

Basic csv reading code added. Next: use llamaparse's own data loaders

In [24]:
# connecting to google drive drive:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# you can get all related files and csv's from here: https://drive.google.com/drive/folders/1a_6BKzeR9W9hDqXYUApfDPPof1w7pzMG

In [25]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os

# The CSV download link
path = '/content/drive/MyDrive/Omdena_Challenge/new_LK_tea_dataset/v2_new_LK_tea_dataset_updated.csv'

# Read the Google Sheet into a pandas DataFrame
all_data = pd.read_csv(path)
all_data.fillna('', inplace=True)
# Display the DataFrame
all_data.head()


,id,class,filename,path,url,language_label,required_ocr,has_tables,text_path,quality,data_origin,retrieved_date_of_issuance,issuing_authority,retrieved_topic,text_content,PDF_or_text,markdown_content,llama_title,llama_issue_date,llama_reference_number
0,0,ACT,ACT_1.txt,,https://www.lawnet.gov.lk/tea-subsidy-3/,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Subsidy – LawNet,"Tea Subsidy\nActs Nos. 12 of 1958, 66 of 1961....",Text,NO_CONTENT_HERE\nThe absorption of certain amo...,Tea Subsidy Act,"19th September, 1958",No. 12 of 1958
1,1,ACT,ACT_2.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Small Holdings Development (Amendment) – L...,|\n\n**Tea Small Holdings Development (Amendme...,Text,# ADVISORY CIRCULAR No_PU 1\n\nIssued in: Febr...,Tea Small Holdings Development (Amendment) Act...,12.08.1997,"Act, No. 21 of 1997"
2,2,ACT,ACT_3.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Small Holdings Development (Amendment) – L...,|\n\n**Tea Small Holdings Development (Amendme...,Text,NO_CONTENT_HERE\n# Systemic Fungicides: at 10 ...,Tea Small Holdings Development (Amendment) Act...,22.10.2003,34 of 2003
3,3,ACT,ACT_4.txt,,https://www.lawnet.gov.lk/tea-small-holdings-d...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Small Holdings Development Law – LawNet,Tea Small Holdings Development Law\nA LAW TO E...,Text,# ADVISORY CIRCULAR No.PU 3\n\nIssued in: Febr...,Tea Small Holdings Development Law,,No. 35 of 1975
4,4,ACT,ACT_5.txt,,https://www.lawnet.gov.lk/tea-research-board-a...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Research Board (Amendment) – LawNet,|\n\n**Tea Research Board (Amendment)** |\n\n|...,Text,NO_CONTENT_HERE\nGrowers may select the mixtur...,Tea Research Board (Amendment) Act,06.11.2006,43 of 2006


In [26]:
# Display the loaded data to check for Tea Control Act which is longer than 50,000 characters
all_data[all_data['retrieved_topic']=='Tea Control – LawNet']

,id,class,filename,path,url,language_label,required_ocr,has_tables,text_path,quality,data_origin,retrieved_date_of_issuance,issuing_authority,retrieved_topic,text_content,PDF_or_text,markdown_content,llama_title,llama_issue_date,llama_reference_number
17,17,ACT,ACT_20.txt,,https://www.lawnet.gov.lk/tea-control-7/,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,,Parliament of Sri Lanka ACT,Tea Control – LawNet,"""Tea Control\nActs Nos. 51 of 1957. 26 of 1962...",Text,# ADVISORY CIRCULAR No.SP 7\n\nIssued in: May ...,Tea Control Act,,51 of 1957


In [27]:
import pandas as pd

# Function to count words and characters in text
def count_words_and_characters(text):
    words = text.split()  # Split by whitespace to get words
    num_words = len(words)
    num_characters = len(text)
    return num_words, num_characters

# Function to count words and characters for the 'text_content' column in the DataFrame
def count_stats_in_dataframe(df):
    total_words = 0
    total_characters = 0

    # Iterate over the 'text_content' column in the DataFrame
    for text in df['text_content'].dropna():  # Drop any NaN values in text_content
        num_words, num_characters = count_words_and_characters(text)
        total_words += num_words
        total_characters += num_characters

    return total_words, total_characters

# Main script
if __name__ == "__main__":

    # Count words and characters
    total_words, total_characters = count_stats_in_dataframe(all_data)

    # Output the results
    print("\nTotal Stats for 'text_content' column in DataFrame:")
    print(f"Total Words: {total_words}")
    print(f"Total Characters: {total_characters}")



Total Stats for 'text_content' column in DataFrame:
Total Words: 179979
Total Characters: 1082140


# 2. Feed all the PDF files as well as ACTs to llamaparse and get parsed chunks in markdown format

for 173 files, around 600 chunks are returned

@TODO convert ACTs via Fast mode (1 cred. / 3 p)
@TODO convert PDFs via Accurate mode (1 cred. / 1 p.), and don't use continuous mode (10 credits)...except for PDFs with Tables....becuase if they span multiple pages, it would be an issue.....................

In [28]:
!pip --quiet install llama-index-core llama-parse llama-index-readers-file python-dotenv

In [29]:
import os
import getpass
# https://cloud.llamaindex.ai/project/d5a5362e-3a4f-4907-8f0b-c8720cd7bb2e/api-key
os.environ['LLAMA_CLOUD_API_KEY'] = getpass.getpass('Enter your LLamacloud API Key: ')
# os.environ['GROQ_API_KEY'] = getpass.getpass('Enter your GROQ API Key: ')

Enter your LLamacloud API Key: ··········


In [30]:
import nest_asyncio
nest_asyncio.apply()

In [31]:
# my version



import pandas as pd
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# Load environment variables (assuming LLAMA_CLOUD_API_KEY is needed somewhere)
load_dotenv()

# Assuming you have your DataFrame `all_data` already loaded
# Example: all_data = pd.read_csv("your_data.csv") or whatever source you're using

# Prepare LlamaParse instance for PDF file that uses accurate mode which is true by default:
# it hallucinates so changing prompt

# pdf_parser = LlamaParse(result_type='text', verbose=True, continuous_mode=True, language= 'en',
#                     parsing_instruction = "You are parsing a regulatory document related to the Sri Lankan Tea Industry. Please give me a json with the title and date of issuance of the document"
#                     )  # "markdown" and "text" are available


# also doesn't work, don't use parsing prompt
pdf_parser = LlamaParse(result_type='markdown', num_workers=8)  # "markdown" and "text" are available

# Prepare LlamaParse instance for text file that uses fast mode:
text_parser = LlamaParse(result_type='markdown', Set_fast_mode = True, num_workers=8)  # "markdown" and "text" are available

# set workers =8 makes it 'fake' parallel

# default is text, modified to markdown for next time. Also, changed number of workers to 8 for next time


In [ ]:
# # works perfectly but no metadata, opened an issue here: https://github.com/run-llama/llama_parse/issues/493
# import pandas as pd
# from llama_parse import LlamaParse
# import asyncio

# # Initialize parsers
# pdf_parser = LlamaParse(result_type='markdown', num_workers=8)
# text_parser = LlamaParse(result_type='markdown', Set_fast_mode=True, num_workers=8)


# # Async function to process PDF documents
# async def process_pdf_documents(pdf_paths):
#     if pdf_paths:  # Only process if there are PDF paths
#         documents = await pdf_parser.aload_data(pdf_paths)
#         return documents
#     return []

# # Async function to process Text documents
# async def process_text_documents(text_paths):
#     if text_paths:  # Only process if there are text paths
#         documents = await text_parser.aload_data(text_paths)
#         return documents
#     return []

# # Async function to process the DataFrame and save to CSV
# async def process_and_save_df(all_data):
#     # Collect file paths for PDFs and Text documents
#     pdf_paths = all_data[all_data['PDF_or_text'] == 'PDF']['path'].tolist()
#     text_paths = all_data[all_data['PDF_or_text'] == 'Text']['text_path'].tolist()

#     # Process PDFs and Texts concurrently
#     pdf_documents = await process_pdf_documents(pdf_paths)
#     text_documents = await process_text_documents(text_paths)

#     # Combine the results
#     all_documents = pdf_documents + text_documents

#     if all_documents:
#         return all_documents
#     else:
#         print("No documents found for processing.")
#         return []

# # Example usage
# async def main():
#     # Assuming `all_data` is a DataFrame that includes columns like 'PDF_or_text', 'path', 'text_path'
#     # all_data = pd.DataFrame({
#     #     'PDF_or_text': ['PDF', 'Text', 'PDF', 'Text'],
#     #     'path': ['./data/file1.pdf', '', './data/file3.pdf', ''],
#     #     'text_path': ['', './data/file2.txt', '', './data/file4.txt']
#     # })

#     # Process documents and get the list of processed documents
#     processed_documents = await process_and_save_df(all_data)

#     if processed_documents:
#         return processed_documents
#     else:
#         print("No documents to process.")

# # Run the asynchronous main function
# processed_documents = asyncio.run(main())


In [32]:
import pandas as pd
from llama_parse import LlamaParse
import asyncio

# Initialize parsers
pdf_parser = LlamaParse(result_type='markdown', num_workers=8)
text_parser = LlamaParse(result_type='markdown', Set_fast_mode=True, num_workers=8)


# Async function to process PDF documents
async def process_pdf_documents(pdf_paths):
    if pdf_paths:  # Only process if there are PDF paths
        file_extractor = {".pdf": pdf_parser}
        pdf_documents = SimpleDirectoryReader(input_files=pdf_paths, file_extractor=file_extractor).load_data()

        return pdf_documents
    return []

# Async function to process Text documents
async def process_text_documents(text_paths):
    if text_paths:  # Only process if there are text paths
        file_extractor = {".txt": text_parser}
        text_documents = SimpleDirectoryReader(input_files=text_paths, file_extractor=file_extractor).load_data()
        return text_documents
    return []

# Async function to process the DataFrame and save to CSV
async def process_and_save_df(all_data):
    # Collect file paths for PDFs and Text documents
    pdf_paths = all_data[all_data['PDF_or_text'] == 'PDF']['path'].tolist()
    text_paths = all_data[all_data['PDF_or_text'] == 'Text']['text_path'].tolist()

    # Process PDFs and Texts concurrently
    pdf_documents = await process_pdf_documents(pdf_paths)
    text_documents = await process_text_documents(text_paths)

    # Combine the results
    all_documents = pdf_documents + text_documents

    if all_documents:
        return all_documents
    else:
        print("No documents found for processing.")
        return []

# Example usage
async def main():
    # Assuming `all_data` is a DataFrame that includes columns like 'PDF_or_text', 'path', 'text_path'
    # all_data = pd.DataFrame({
    #     'PDF_or_text': ['PDF', 'Text', 'PDF', 'Text'],
    #     'path': ['./data/file1.pdf', '', './data/file3.pdf', ''],
    #     'text_path': ['', './data/file2.txt', '', './data/file4.txt']
    # })

    # Process documents and get the list of processed documents
    processed_documents = await process_and_save_df(all_data)

    if processed_documents:
        return processed_documents
    else:
        print("No documents to process.")

# Run the asynchronous main function
processed_documents = asyncio.run(main())


Started parsing the file under job_id a037f7f5-8957-40d0-bb86-ea709134643e
Started parsing the file under job_id 6606e460-04aa-4cf7-8a2b-cf755ee75644
Started parsing the file under job_id fa0b1197-9715-4d3a-b1b7-1c62188d7698
Started parsing the file under job_id c723d952-3d3d-43c3-9717-aa447fba9cd2
Started parsing the file under job_id 46c02f34-2049-40f8-98bc-60c5b621a957
Started parsing the file under job_id 496004f6-a267-438e-b7e7-9a69bc291bb2
Started parsing the file under job_id 455060b7-32ac-472a-868a-8f6e59c899c4
Started parsing the file under job_id 7ddd2527-79ea-4691-a154-df4b7cc8cfd0
Started parsing the file under job_id c15e6f94-27e0-4b4b-bf43-67495a0af37b
Started parsing the file under job_id c03baad1-c50a-48e0-b77a-9336d9e38dd6
Started parsing the file under job_id 6619392f-3fa2-4858-b9a0-164adf918c92
Started parsing the file under job_id 6154d308-f03d-402a-8560-3905868714e6
Started parsing the file under job_id 6e219252-401e-4438-ae1e-3b43fd72425f
Started parsing the file 

In [33]:
processed_documents[0]

Document(id_='efdc9ce0-2187-4322-9e49-86631cb520e2', embedding=None, metadata={'file_path': '/content/drive/MyDrive/Omdena_Challenge/new_LK_tea_dataset/Circulers/07_General-Instruction-for-the-Submission-of-Tea-Samples-for-Testing_compressed.pdf', 'file_name': '07_General-Instruction-for-the-Submission-of-Tea-Samples-for-Testing_compressed.pdf', 'file_type': 'application/pdf', 'file_size': 14710, 'creation_date': '2024-11-19', 'last_modified_date': '2024-11-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\n\n# GENERAL INSTRUCTION FOR THE SUBMISSION OF TEA SAMPLES FOR TESTING\n\n# 1. Submission of tea sample\n\n- Sample of tea should be submitted to the Analytical Laboratory on any working day (Monday-Fri

Attributes to Extract:

id_: Unique identifier for the document.
embedding: An optional field, which may be None if not set.
metadata: A dictionary containing any associated metadata (in this case, it's empty).
relationships: A dictionary containing any relationships (empty in this case).
text: The content of the document itself.
mimetype: The MIME type of the document (in this case, it's 'text/plain').
start_char_idx, end_char_idx: These are None in your example, but may hold character index information if applicable.

Optional:

metadata_template, text_template, and other fields can be ignored if not needed.

In [34]:
len(processed_documents)

587

In [35]:
type(processed_documents)

list

In [36]:
type(processed_documents[0])

llama_index.core.schema.Document

In [37]:
import pandas as pd
import json
from llama_index.core.schema import Document
import csv

# Sample function to convert a single Document to a dictionary
def document_to_dict(document: Document):
    doc_dict = {
        "id": document.id_,
        "embedding": document.embedding,  # May be None
        "metadata": document.metadata,  # Empty in your case
        "excluded_embed_metadata_keys": document.excluded_embed_metadata_keys,
        "excluded_llm_metadata_keys": document.excluded_llm_metadata_keys,
        "relationships": document.relationships,  # Empty in your case
        "text": document.text,  # The markdown content
        "mimetype": document.mimetype,
        "start_char_idx": document.start_char_idx,
        "end_char_idx": document.end_char_idx,
        "text_template": document.text_template,
        "metadata_template": document.metadata_template,
        "metadata_seperator": document.metadata_seperator
    }
    return doc_dict

# Convert the list of Document objects to a list of dictionaries
def documents_to_dicts(documents: list):
    return [document_to_dict(doc) for doc in documents]

# Convert the list of dictionaries to a pandas DataFrame
def documents_to_dataframe(documents: list):
    # Convert documents list to dicts
    doc_dicts = documents_to_dicts(documents)

    # Convert list of dictionaries to a DataFrame
    df = pd.DataFrame(doc_dicts)
    return df


processed_df = documents_to_dataframe(processed_documents)


# Show the resulting DataFrame (markdown text will be preserved)
processed_df




,id,embedding,metadata,excluded_embed_metadata_keys,excluded_llm_metadata_keys,relationships,text,mimetype,start_char_idx,end_char_idx,text_template,metadata_template,metadata_seperator
0,efdc9ce0-2187-4322-9e49-86631cb520e2,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
1,cfb288e1-c5fb-4ac8-aed6-78346c5b730b,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
2,c9d02201-d81e-4394-baf3-272df51aee10,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},NO_CONTENT_HERE,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
3,2026d9d2-3221-459e-aa8f-0cfd80c852de,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# Sri Lanka Tea Board Minimum Quality Standard...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
4,4024786c-5d03-4832-83ea-db6652110139,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# Sri Lanka Tea Board Minimum Quality Standard...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,42857a8a-fd62-4c11-811b-148169fafdb5,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# Definitions\n\n” Minister ” means the Minist...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
583,03cdee3f-2896-4d83-8f3b-51004df2d838,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# PARLIAMENT OF THE DEMOCRATIC SOCIALIST REPUB...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
584,e20a1a9f-3f0b-4489-846d-44f841ea3000,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},remuneration as may be specified in the releva...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n
585,0844a308-bf84-482d-b780-a8652d14c374,None,{'file_path': '/content/drive/MyDrive/Omdena_C...,"[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},# PARLIAMENT OF THE DEMOCRATIC SOCIALIST REPUB...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n


In [38]:
# Sample function to save the DataFrame to CSV with markdown preservation
def save_dataframe_to_csv(df, filename):
    df.to_csv(filename,
              index=False,  # Do not write row indices
              quoting=csv.QUOTE_MINIMAL,  # Quote fields that contain special characters
              quotechar='"',  # Use double quotes to wrap fields
              escapechar='\\',  # Escape any quotes inside markdown content
              lineterminator='\n',  # Make sure to preserve newlines in markdown content
              encoding='utf-8')  # Ensure proper encoding to handle special characters in markdown



# Save the DataFrame to a CSV file with markdown content preserved
save_dataframe_to_csv(processed_df, '/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llama_processed_documents_markdown.csv')


OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING'

# Extract metadata


In [ ]:
# Read the DataFrame (assuming the 6th column contains dictionaries with the 'text' key)
processed_df = pd.read_csv('/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llama_processed_documents_markdown.csv')
processed_df

,id,embedding,metadata,excluded_embed_metadata_keys,excluded_llm_metadata_keys,relationships,text,mimetype,start_char_idx,end_char_idx,text_template,metadata_template,metadata_seperator
0,9753d24f-7cd5-43a2-8cde-7e3e12666a54,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
1,b69c23a5-53d4-49ca-9411-30b3be1f4a66,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
2,7afcefa7-7e90-4cae-8b16-0d8f7b145c0c,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},NO_CONTENT_HERE,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
3,b1a04e4c-1751-4656-bd6b-1c401bfd992f,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# Sri Lanka Tea Board Minimum Quality Standard...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
4,b72360c5-cf47-4b95-ad8e-01aa32e4d996,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# Sri Lanka Tea Board Minimum Quality Standard...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,179ffe9c-45e7-4b29-9481-7896300971e5,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# Definitions\n\n” Minister ” means the Minist...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
603,4abbdb31-0473-455b-b756-f460f95de9ef,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# PARLIAMENT OF THE DEMOCRATIC SOCIALIST REPUB...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
604,1b1c97b5-0b0d-4006-94db-146c85a316c8,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},remuneration as may be specified in the releva...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n
605,537067c4-f5b6-47f4-9582-5f69f3542460,NaN,{'file_path': '/content/drive/MyDrive/Omdena_C...,"['file_name', 'file_type', 'file_size', 'creat...","['file_name', 'file_type', 'file_size', 'creat...",{},# PARLIAMENT OF THE DEMOCRATIC SOCIALIST REPUB...,text/plain,NaN,NaN,{metadata_str}\n\n{content},{key}: {value},\n


In [ ]:
import ast
# Convert the 'metadata' column from string representation to dictionary
processed_df['metadata'] = processed_df['metadata'].apply(ast.literal_eval)

# Now you can access dictionary keys
first_file_name = processed_df['metadata'].iloc[0]['file_name']
print(first_file_name)  # Output: file1.txt


07_General-Instruction-for-the-Submission-of-Tea-Samples-for-Testing_compressed.pdf


In [ ]:
import pandas as pd

# Creating the DataFrame with 'metadata' column containing dictionaries
df_metadata = pd.DataFrame()
# Extracting the 'filename' from the 'metadata' column
df_metadata['file_name'] = processed_df['metadata'].apply(lambda x: x.get('file_name', None))
df_metadata['markdown_content'] = processed_df['text']

# Display the DataFrame
print(df_metadata)


                                             file_name  \
0    07_General-Instruction-for-the-Submission-of-T...   
1    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
2    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
3    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
4    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
..                                                 ...   
602                                         ACT_27.txt   
603                                         ACT_28.txt   
604                                         ACT_28.txt   
605                                         ACT_29.txt   
606                                         ACT_29.txt   

                                      markdown_content  
0    # ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...  
1    # MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...  
2                                      NO_CONTENT_HERE  
3    # Sri Lanka Tea Board Minimum Quality Standard...  
4    # Sri Lanka T

# only pick smaller mardown text chunks, first and last two in a separate dataframe

In [ ]:
import pandas as pd

# Group by 'file_name' and apply a custom function to each group
def merge_markdown_content(group):
    # Combine all markdown content in the group with line breaks
    combined_markdown_all = "\n".join(group['markdown_content'])

    # Take the first row's markdown content
    first_row = group.iloc[0]

    # Take the last two rows (if there are at least two)
    last_two_rows = group.tail(2)

    # Combine the first row text_content with the last two rows text_content
    combined_markdown_first_last = first_row['markdown_content'] + " " + " ".join(last_two_rows['markdown_content'])

    # Return a new series with 'file_name' and the combined markdown content
    return pd.Series({
        'file_name': first_row['file_name'],  # Keeping 'file_name' from the first row
        'markdown_content_long': combined_markdown_all,
        'markdown_content_short': combined_markdown_first_last
    })

# Apply the function to each group
# To avoid deprecation warning, use `as_index=True` (default) and reset index after `apply`
merged_df = df_metadata.groupby('file_name', as_index=False).apply(merge_markdown_content)

# Drop the multi-index that `apply` creates (this is important to avoid a hierarchical index)
merged_df = merged_df.reset_index(drop=True)
update_markdown= merged_df.copy()
# View the merged DataFrame
merged_df


                                             file_name  \
0    01_Sri-Lanka-Tea-Board-Standards-Directives-fo...   
1    07_General-Instruction-for-the-Submission-of-T...   
2                                       1998 10 30.pdf   
3                                        2158-03_E.pdf   
4                                        2245-29_E.pdf   
..                                                 ...   
162                           al-mqs-08 2008 04 24.pdf   
163  guidelines_on_covid-19_in_tea_plantations_eng_...   
164                                      mf bl 119.pdf   
165                                       scan0002.pdf   
166             tc - e - vi - 68 -vol 2 2003 03 07.pdf   

                                 markdown_content_long  \
0    # MINISTRY OF PLANTATION\n\n# SRI LANKA TEA BO...   
1    # ANALYTICAL LABORATORY OF SRILANKA TEA BOARD\...   
2    # Sri Lanka Tea Board\n\n574, Galle Road, Colo...   
3    # PART I : SEC. (I) - GAZETTE EXTRAORDINARY OF...   
4    # PART I

<ipython-input-268-f52080032f58>:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = df_metadata.groupby('file_name', as_index=False).apply(merge_markdown_content)


# save markdown_content_long to md files in file directory?

# Get metadata via groq

In [ ]:
!pip -q install groq

In [ ]:
import os
import getpass
# os.environ['LLAMA_CLOUD_API_KEY'] = getpass.getpass('Enter your LLamacloud API Key: ')


os.environ['GROQ_API_KEY'] = getpass.getpass('Enter your GROQ API Key: ')
groq_api=os.environ['GROQ_API_KEY']

Enter your GROQ API Key: ··········


In [ ]:
import pandas as pd
import json
from groq import Groq
from google.colab import userdata
import os
import re

# Assuming Groq API key is set up properly


client = Groq(api_key=groq_api)

def clean_text_for_metadata_extraction(text):
    """
    Clean and preprocess the extracted text content to make it suitable for metadata extraction.
    This will remove unwanted text such as 'Copyright' and any excessive spaces or line breaks.
    """
    if not isinstance(text, str):
        return None

    # Remove specific unwanted text like 'Copyright' (case-insensitive) and everything after it
    text = re.sub(r'copyright[\s\S]*?(\n|\r|\Z)', '', text, flags=re.IGNORECASE)  # Remove copyright section and text after it

    return text

def extract_metadata(row):
    """
    Extract title and published date from the document's text using the Groq API.
    The prompt is designed to instruct the Groq model to return a JSON object containing these two pieces of information.
    """
    text =  row['markdown_content_short']

    # Clean and preprocess the text before passing to the Groq API
    cleaned_text = clean_text_for_metadata_extraction(text)
    if not cleaned_text:
        return None

    prompt = (
        "Please process the following document and only extract the following details in English, "
        "in valid JSON format:\n\n"
        "- parsed_title: The title of the document (if available, otherwise return 'None').\n"
        "- parsed_issue_date: The published date of the document (if available, otherwise return 'None').\n"
        "- parsed_reference_number: The reference number of the document (if available, otherwise return 'None').\n\n"
        "Document:\n\n"
        f"{cleaned_text}\n\n"
        "Example output format:\n"
        '{"parsed_title": "<extracted title or None>", "parsed_issue_date": "<extracted date or None>", "parsed_reference_number": "<extracted reference number or None>"}\n'
    )


    # Send the prompt to the Groq API
    try:
        completion = client.chat.completions.create(
            # model="llama3-70b-8192",
            # model="llama3-8b-8192",
            model = "llama-3.1-8b-instant", # Make sure this model is appropriate for your task
            messages=[
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": "```json"}
            ],
            stop="```",
        )

        # Try to extract the JSON response
        return completion.choices[0].message.content.strip("```json").strip()

    except (AttributeError, IndexError) as e:
        print(f"Error processing row: {e}")
        return None

# Apply the metadata extraction to the DataFrame
merged_df['metadata_returned'] = merged_df.apply(extract_metadata, axis = 1)


merged_df

In [ ]:
# Sample function to save the DataFrame to CSV with markdown preservation
def save_dataframe_to_csv(df, filename):
    df.to_csv(filename,
              index=False,  # Do not write row indices
              quoting=csv.QUOTE_MINIMAL,  # Quote fields that contain special characters
              quotechar='"',  # Use double quotes to wrap fields
              escapechar='\\',  # Escape any quotes inside markdown content
              lineterminator='\n',  # Make sure to preserve newlines in markdown content
              encoding='utf-8')  # Ensure proper encoding to handle special characters in markdown



# Save the DataFrame to a CSV file with markdown content preserved
save_dataframe_to_csv(merged_df, '/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llama_parsed_metadata_instant.csv')


In [ ]:
## just a local backup copy becuase it can take hours to get results (30 min in this case)
bu_parsed_df = merged_df.copy()

In [ ]:
merged_df = bu_merged_df.copy()

In [ ]:
# or read from from csv
merged_df = pd.read_csv('/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llama_parsed_metadata_instant.csv')
merged_df

In [ ]:
## for llama 70b
# # Using iloc to get the first row's metadata column value
# metadata_test_string = merged_df.iloc[0]['metadata_returned']
# metadata_test_string

In [ ]:
#  ## for llama 70b
#  parsed_dict = ast.literal_eval(metadata_test_string)
#  parsed_dict

In [ ]:
# ## for llama 70b

# import pandas as pd
# import json

# # Sample function to clean and parse metadata string
# def parse_json_from_string(metadata_string):
#     try:
#         parsed_dict = ast.literal_eval(metadata_string)
#         return parsed_dict  # Return the parsed JSON object
#     except Exception as e:
#         print(f"Error processing string as dict: {e}")
#         return None

# # Assuming your DataFrame is called df and the metadata is in a column called 'metadata'

# # Apply the parsing function to the 'metadata' column
# merged_df['parsed_metadata'] = merged_df['metadata_returned'].apply(parse_json_from_string)

# # # Convert the parsed JSON string into actual JSON objects (list of dictionaries)
# # # merged_df['parsed_metadata'] = merged_df['parsed_metadata'].apply(json.loads)

# # # Expand the JSON objects into separate columns
# expanded_columns = merged_df['parsed_metadata'].apply(pd.Series)

# # # Merge the expanded columns back into the original DataFrame
# merged_df = pd.concat([merged_df, expanded_columns], axis=1)

# # # Drop the original 'parsed_metadata' column if it's no longer needed
# merged_df.drop(columns=['parsed_metadata', 'metadata_returned','parsed_json', 'signature'], inplace=True)

# # Display the updated DataFrame
# merged_df


In [ ]:
# ## for llama 70b

# # Sample function to save the DataFrame to CSV with markdown preservation
# def save_dataframe_to_csv(df, filename):
#     df.to_csv(filename,
#               index=False,  # Do not write row indices
#               quoting=csv.QUOTE_MINIMAL,  # Quote fields that contain special characters
#               quotechar='"',  # Use double quotes to wrap fields
#               escapechar='\\',  # Escape any quotes inside markdown content
#               lineterminator='\n',  # Make sure to preserve newlines in markdown content
#               encoding='utf-8')  # Ensure proper encoding to handle special characters in markdown



# # Save the DataFrame to a CSV file with markdown content preserved
# save_dataframe_to_csv(merged_df, '/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/llama_parsed_metadata_70b.csv')


# append metadata results to main csv

In [ ]:
# Rename columns in the DataFrame
merged_df = merged_df.rename(columns={
    'file_name':'filename',
    'markdown_content_long': 'markdown_content',
    'parsed_title': 'llama_title',
    'parsed_issue_date': 'llama_issue_date',
    'parsed_reference_number': 'llama_reference_number'
})

In [ ]:
def append_merged_df_to_all_data(all_data, merged_df):
  """
  Appends the merged_df columns based on the 'file_name' to the all_data DataFrame.
  """
  # all_data = all_data.merge(merged_df[['file_name', 'markdown_content_long', 'parsed_title','parsed_issue_date', 'parsed_reference_number']], on='file_name', how='left')
  all_data = all_data.merge(merged_df[['filename', 'markdown_content', 'llama_title','llama_issue_date', 'llama_reference_number']], on='filename', how='left')

  return all_data


# Append merged_df columns to all_data
all_data = append_merged_df_to_all_data(all_data, merged_df)

# Print to confirm
print(all_data.head())

  class   filename path                                                url  \
0   ACT  ACT_1.txt                https://www.lawnet.gov.lk/tea-subsidy-3/   
1   ACT  ACT_2.txt       https://www.lawnet.gov.lk/tea-small-holdings-d...   
2   ACT  ACT_3.txt       https://www.lawnet.gov.lk/tea-small-holdings-d...   
3   ACT  ACT_4.txt       https://www.lawnet.gov.lk/tea-small-holdings-d...   
4   ACT  ACT_5.txt       https://www.lawnet.gov.lk/tea-research-board-a...   

  language_label required_ocr has_tables  \
0             en           no         no   
1             en           no         no   
2             en           no         no   
3             en           no         no   
4             en           no         no   

                                           text_path quality data_origin  \
0  /content/drive/MyDrive/Omdena_Challenge/new_LK...    good     scraped   
1  /content/drive/MyDrive/Omdena_Challenge/new_LK...    good     scraped   
2  /content/drive/MyDrive/Omdena_Chall

In [ ]:
all_data.columns

Index(['class', 'filename', 'path', 'url', 'language_label', 'required_ocr',
       'has_tables', 'text_path', 'quality', 'data_origin',
       'retrieved_date_of_issuance', 'issuing_authority', 'retrieved_topic',
       'text_content', 'PDF_or_text', 'markdown_content', 'llama_title',
       'llama_issue_date', 'llama_reference_number'],
      dtype='object')

In [ ]:
len(all_data)

170

# get rows with NO_CONTENT_HERE, and mark as bad quality in original csv, and remove for next step in next code

In [ ]:
# prompt: get markdown content rows where only NO CONTENT HERE exists and remove the rows

def remove_no_content_rows(df):
  """Removes rows from a DataFrame where 'markdown_content_short' contains only 'NO CONTENT HERE'."""
  df = df[~df['markdown_content'].str.contains(r'^NO_CONTENT_HERE$', na=False)]
  return df

# Assuming 'merged_df' is the DataFrame with the 'markdown_content_short' column
all_data = remove_no_content_rows(all_data)

# Now 'merged_df' has rows with only 'NO CONTENT HERE' removed.

# save main csv

In [ ]:
# Sample function to save the DataFrame to CSV with markdown preservation
def save_dataframe_to_csv(df, filename):
    df.to_csv(filename,
              index=False,  # Do not write row indices
              quoting=csv.QUOTE_MINIMAL,  # Quote fields that contain special characters
              quotechar='"',  # Use double quotes to wrap fields
              escapechar='\\',  # Escape any quotes inside markdown content
              lineterminator='\n',  # Make sure to preserve newlines in markdown content
              encoding='utf-8')  # Ensure proper encoding to handle special characters in markdown



# Save the DataFrame to a CSV file with markdown content preserved
save_dataframe_to_csv(all_data, '/content/drive/MyDrive/Omdena_Challenge/PREPROCESSING/v2_new_LK_tea_dataset_updated.csv')


# estimating number of token from markdown file

In [ ]:
import pandas as pd

# Function to count words and characters in text
def count_words_and_characters(text):
    words = text.split()  # Split by whitespace to get words
    num_words = len(words)
    num_characters = len(text)
    return num_words, num_characters

# Function to count words and characters for the 'text_content' column in the DataFrame
def count_stats_in_dataframe(df):
    total_words = 0
    total_characters = 0

    # Iterate over the 'text_content' column in the DataFrame
    for text in df['markdown_content'].dropna():  # Drop any NaN values in text_content
        num_words, num_characters = count_words_and_characters(text)
        total_words += num_words
        total_characters += num_characters

    return total_words, total_characters

# Main script
if __name__ == "__main__":

    # Count words and characters
    total_words, total_characters = count_stats_in_dataframe(all_data)

    # Output the results
    print("\nTotal Stats for 'markdown_content' column in DataFrame:")
    print(f"Total Words: {total_words}")
    print(f"Total Characters: {total_characters}")
